# Clustering Crypto

In [25]:
# Initial imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path

### Fetching Cryptocurrency Data

In [26]:
# Use the following endpoint to fetch json data
#url = "https://min-api.cryptocompare.com/data/all/coinlist"
#r = requests.get(url)
#crypto_data = r.json()['Data']
file_path = Path("./Resources/crypto_data.csv")

In [27]:
# Create a DataFrame 
#crypto_df = pd.DataFrame(crypto_data).T
crypto_df = pd.read_csv(file_path)
crypto_df.set_index('Unnamed: 0', inplace=True)
crypto_df.index.names = ['Crypto Index']
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Crypto Index,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [28]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]

In [29]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df.IsTrading.eq(True)]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Crypto Index,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [32]:
# Keep only cryptocurrencies with a working algorithm
crypto_df= crypto_df[crypto_df.Algorithm != 'N/A']

In [33]:
# Remove the "IsTrading" column
crypto_df.drop(['IsTrading'], axis = 1, inplace=True)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Crypto Index,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [34]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True)

In [35]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df= crypto_df[crypto_df.TotalCoinsMined != 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Crypto Index,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [36]:
# Drop rows where there are 'N/A' text values
crypto_df= crypto_df[crypto_df.ProofType != 'N/A']
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Crypto Index,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [37]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coinname_df = crypto_df.iloc[:,0]
coinname_df

Crypto Index
42          42 Coin
404         404Coin
1337      EliteCoin
BTC         Bitcoin
ETH        Ethereum
           ...     
ZEPH         ZEPHYR
GAP         Gapcoin
BDX          Beldex
ZEN         Horizen
XBC     BitcoinPlus
Name: CoinName, Length: 533, dtype: object

In [13]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
clean_crypto_df = crypto_df.drop(columns='CoinName')
clean_crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Crypto Index,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610


In [14]:
# Create dummy variables for text features
X = pd.get_dummies(clean_crypto_df, columns=['Algorithm', 'ProofType'])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Crypto Index,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Standardize data
data_scaler = StandardScaler()
data_scaler.fit(X)
X_scaled = data_scaler.transform(X)
X_scaled[:1]

array([[-0.11674788, -0.15286468, -0.0433555 , -0.0433555 , -0.0433555 ,
        -0.06137164, -0.07523548, -0.0433555 , -0.06137164, -0.06137164,
        -0.0433555 , -0.0433555 , -0.19226279, -0.06137164, -0.09731237,
        -0.0433555 , -0.11536024, -0.07523548, -0.0433555 , -0.0433555 ,
        -0.15176505, -0.0433555 , -0.13105561, -0.0433555 , -0.0433555 ,
        -0.08695652, -0.0433555 , -0.0433555 , -0.0433555 , -0.0433555 ,
        -0.06137164, -0.0433555 , -0.08695652, -0.08695652, -0.08695652,
        -0.0433555 , -0.13105561, -0.13827675, -0.13827675, -0.0433555 ,
        -0.06137164, -0.0433555 , -0.07523548, -0.1815096 , -0.0433555 ,
        -0.0433555 , -0.0433555 , -0.07523548, -0.15811388, -0.3145935 ,
        -0.0433555 , -0.08695652, -0.07523548, -0.06137164, -0.0433555 ,
         1.38873015, -0.0433555 , -0.0433555 , -0.06137164, -0.0433555 ,
        -0.0433555 , -0.0433555 , -0.0433555 , -0.0433555 , -0.0433555 ,
        -0.0433555 , -0.0433555 , -0.39836623, -0.0

### Reducing Dimensions Using PCA

In [16]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X_scaled)

In [17]:
# Create a DataFrame with the principal components data
df_crypto_pca = pd.DataFrame( data=crypto_pca, columns=["principal component 1", 
                              "principal component 2", 
                              "principal component 3"], index=clean_crypto_df.index
)
df_crypto_pca

,principal component 1,principal component 2,principal component 3
Crypto Index,,,
42,-0.327762,1.039815,-0.304042
404,-0.311093,1.040285,-0.304374
1337,2.312430,1.554226,-0.377272
BTC,-0.145344,-1.363073,0.136551
ETH,-0.152966,-2.030062,0.239669
...,...,...,...
ZEPH,2.482824,0.552567,-0.011011
GAP,-0.325807,1.039741,-0.304059
BDX,0.327703,-2.286296,0.222517


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [18]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
# Looking for the best k
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Create the Elbow Curve using hvPlot
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [19]:
# Initialize the K-Means model
crypto_model = KMeans(n_clusters=4, random_state=5)
# Fit the model
crypto_model.fit(df_crypto_pca)
# Predict clusters
predictions = crypto_model.predict(df_crypto_pca)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
final_crpyto_table = pd.concat([crypto_df, df_crypto_pca], axis = 1)
#added Class 
final_crpyto_table["Class"] = crypto_model.labels_
final_crpyto_table['TotalCoinSupply'] = final_crpyto_table['TotalCoinSupply'].astype(float, errors = 'raise')
hv_crypto = final_crpyto_table
final_crpyto_table.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,Class
Crypto Index,,,,,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.327762,1.039815,-0.304042,0
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.311093,1.040285,-0.304374,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.312430,1.554226,-0.377272,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.145344,-1.363073,0.136551,1
ETH,Ethereum,Ethash,PoW,1.076842e+08,0.000000e+00,-0.152966,-2.030062,0.239669,1


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [20]:
# Scale data to create the scatter plot
#create scaled version by taking the ln() of the variables in question for scaling correctly. this turns this into an exponetial chart
init_value_1 = 0
init_value_2 = 0
Scaled_Total_Coin_Supply = []
Scaled_Total_Coins_Mined = []
for i in range(len(final_crpyto_table)):
    init_value_1 = final_crpyto_table.iloc[i,
                                             final_crpyto_table.columns.get_loc('TotalCoinSupply')
                                               ]
    if init_value_1 != 0:
        Scaled_Total_Coin_Supply.append(np.log(init_value_1))
    else:
        Scaled_Total_Coin_Supply.append(init_value_1)
        
    init_value_2 = final_crpyto_table.iloc[i,
                                                final_crpyto_table.columns.get_loc('TotalCoinsMined')
                                               ]
    if init_value_2 != 0:
        Scaled_Total_Coins_Mined.append(np.log(init_value_2))
    else:
        Scaled_Total_Coins_Mined.append(init_value_2)
    
    
            
final_crpyto_table['ScaledTotalCoinsMined'] = Scaled_Total_Coins_Mined
final_crpyto_table['ScaledTotalCoinSupply'] = Scaled_Total_Coin_Supply

final_crpyto_table.head()

/home/rex/anaconda3/envs/pyvisenv/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in log


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,Class,ScaledTotalCoinsMined,ScaledTotalCoinSupply
Crypto Index,,,,,,,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.327762,1.039815,-0.304042,0,3.737669,3.737670
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.311093,1.040285,-0.304374,0,20.776982,20.092154
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.312430,1.554226,-0.377272,0,24.100151,26.473166
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.145344,-1.363073,0.136551,1,16.701828,16.860033
ETH,Ethereum,Ethash,PoW,1.076842e+08,0.000000e+00,-0.152966,-2.030062,0.239669,1,18.494714,0.000000


In [21]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
final_crpyto_table.hvplot.scatter(x='ScaledTotalCoinsMined',
                                         y='ScaledTotalCoinSupply',
                                         by='Class',
                                         xlabel= 'Ln(TotalCoinsMined)',
                                         ylabel= 'Ln(TotaCoinSupply)',
                                         hover_cols=['CoinName'])

:NdOverlay   [Class]
   :Scatter   [ScaledTotalCoinsMined]   (ScaledTotalCoinSupply,CoinName)

#### Table of Tradable Cryptocurrencies

In [22]:
# Table with tradable cryptos
final_crpyto_table.hvplot.table(columns=['CoinName',
                                         'Algorithm',
                                         'ProofType',
                                         'TotalCoinsMined',
                                         'TotalCoinSupply',
                                         'Class'], 
                                sortable=True, 
                                selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,Class]

In [38]:
# Print the total number of tradable cryptocurrencies
print(f'there are a total number of {len(final_crpyto_table)} tradable crypto currencies')

there are a total number of 533 tradable crypto currencies
